In [13]:
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import pandas as pd
import polars as pl
import polars_st as st
import pyogrio
from electoralyze import region
from electoralyze.common import constants
from electoralyze.common.geometry import to_geopandas, to_geopolars
from electoralyze.common.testing.region_fixture import RegionMocked, read_true_geometry, read_true_metadata
from electoralyze.region.redistribute.mapping import (
    _create_intersection_area_mapping,
    _get_remaining_area,
    create_region_mapping_base,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# read_true_geometry("region_a").pipe(to_geopandas).explore()

In [27]:
intersection_area = _create_intersection_area_mapping(
    geometry_from=read_true_geometry("region_a"),
    geometry_to=read_true_geometry("region_b"),
)
intersection_area.pipe(print)

shape: (10, 3)
┌──────────┬──────────┬───────────────────┐
│ region_a ┆ region_b ┆ intersection_area │
│ ---      ┆ ---      ┆ ---               │
│ str      ┆ str      ┆ f64               │
╞══════════╪══════════╪═══════════════════╡
│ M        ┆ A        ┆ 0.25              │
│ M        ┆ B        ┆ 0.75              │
│ N        ┆ A        ┆ 0.25              │
│ N        ┆ C        ┆ 0.75              │
│ O        ┆ A        ┆ 0.75              │
│ O        ┆ B        ┆ 0.25              │
│ P        ┆ A        ┆ 0.75              │
│ P        ┆ C        ┆ 0.25              │
│ null     ┆ C        ┆ 1.0               │
│ null     ┆ B        ┆ 1.0               │
└──────────┴──────────┴───────────────────┘


In [29]:
intersection_area.to_dict(as_series=False)

{'region_a': ['M', 'M', 'N', 'N', 'O', 'O', 'P', 'P', None, None],
 'region_b': ['A', 'B', 'A', 'C', 'A', 'B', 'A', 'C', 'C', 'B'],
 'intersection_area': [0.25,
  0.75,
  0.25,
  0.75,
  0.75,
  0.25,
  0.75,
  0.25,
  1.0,
  1.0]}